# Marginal likelihood timing tests

In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import starry
import theano
import theano.tensor as tt

starry.config.lazy = True

In [ ]:
np.random.seed(0)
map = starry.Map(ydeg=10)
inc_true = 60.0
map.inc = inc_true
map[1:, :] = np.random.randn(map.Ny - 1)
npts = 1000
sigma = 0.05
theta = np.linspace(0, 360, npts)
flux = map.flux(theta=theta).eval()
flux += np.random.randn(npts) * sigma

In [ ]:
plt.plot(theta, flux)
plt.xlabel(r"$\theta$ [deg]")
plt.ylabel(r"flux [arbitrary units]");

## Compute the posterior over inclination

In [ ]:
map.set_data(flux, C=sigma ** 2)
map.set_prior(L=1)


def _lnlike(inc):
    map.inc = inc
    return map.lnlike(theta=theta)


inc = tt.dscalar()
lnlike = theano.function([inc], _lnlike(inc), profile=True)

In [ ]:
incs = np.linspace(0, 90, 50)
ll_val = np.array([lnlike(inc) for inc in incs])

like = np.exp(ll_val - ll_val.max())
plt.plot(incs, like)
plt.xlabel(r"$\theta$ [deg]")
plt.ylabel(r"posterior probability")
plt.axvline(inc_true, color="C1", label="true")
plt.legend();

## Timing tests

In [ ]:
def get_func_and_grad(C, L, woodbury):

    map.set_data(flux, C=C)
    map.set_prior(L=L)

    def _lnlike(inc):
        map.inc = inc
        return map.lnlike(theta=theta, woodbury=woodbury)

    # Compile the theano functions
    inc = tt.dscalar()
    func = theano.function([inc], _lnlike(inc), profile=True)
    grad = theano.function([inc], theano.grad(_lnlike(inc), inc), profile=True)
    return func, grad

In [ ]:
import itertools

In [ ]:
C = [np.array(sigma ** 2), sigma ** 2 * np.ones(npts), sigma ** 2 * np.eye(npts)]
L = [np.array(1.0), np.ones(map.Ny - 1), np.eye(map.Ny - 1)]
woodbury = [False, True]
params = list(itertools.product(C, L, woodbury))

In [ ]:
ms = list(itertools.product(C, L, woodbury))

In [ ]:
def color(param):
    if type(param) is bool:
        if param is True:
            return "w"
        else:
            return "k"
    elif param.ndim == 0:
        return "C0"
    elif param.ndim == 1:
        return "C1"
    else:
        return "C2"


c1 = []
c2 = []
c3 = []
for param in params:
    c1.append(color(param[0]))
    c2.append(color(param[1]))
    c3.append(color(param[2]))

In [ ]:
from tqdm.notebook import tqdm
import time

In [ ]:
def time_func_and_grad(*args, nruns=10):
    func, grad = get_func_and_grad(*args)
    tfunc = np.zeros(nruns)
    tgrad = np.zeros(nruns)
    for k in range(nruns):
        tstart = time.time()
        func(inc_true)
        tfunc[k] = time.time() - tstart
        tstart = time.time()
        grad(inc_true)
        tgrad[k] = time.time() - tstart
    tfunc = np.median(tfunc)
    tgrad = np.median(tgrad)
    return tfunc, tgrad


tfunc = np.zeros(len(params))
tgrad = np.zeros(len(params))
for i, args in tqdm(enumerate(params), total=len(params)):
    tfunc[i], tgrad[i] = time_func_and_grad(*args)

In [ ]:
plt.scatter(tfunc, tgrad, c=c1, ec="k", s=300)
plt.scatter(tfunc, tgrad, c=c2, ec="k", s=150)
plt.scatter(tfunc, tgrad, c=c3, ec="k", s=50)
plt.xscale("log")
plt.yscale("log");